<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PostgreSQL_03B_SQL_Level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

PostgreSQL Tutorial https://www.w3resource.com/PostgreSQL/tutorial.php

#Setup

In [1]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [2]:
#!pip install psycopg2
import psycopg2
import pandas as pd

In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis


ALTER ROLE
ALTER ROLE


In [4]:
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:dbpraxis:upraxis:upass


## Create and Load Tables

In [5]:
# Create Fact Table using command line interface
#
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c \
"drop table if exists ss_order;\
CREATE TABLE IF NOT EXISTS ss_order (\
    RowID smallint,\
    OrderID char(14),OrderDate date,\
    ShipDate date,ShipMode varchar(16),\
    CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
    Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
    ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
    Sales decimal(8,2), Quantity smallint, Discount decimal(4,2), Profit decimal(8,2) \
); \
"

NOTICE:  table "ss_order" does not exist, skipping
CREATE TABLE


In [6]:
#Describe the table ss_order
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\d+ ss_order'

                                                   Table "public.ss_order"
    Column    |          Type          | Collation | Nullable | Default | Storage  | Compression | Stats target | Description 
--------------+------------------------+-----------+----------+---------+----------+-------------+--------------+-------------
 rowid        | smallint               |           |          |         | plain    |             |              | 
 orderid      | character(14)          |           |          |         | extended |             |              | 
 orderdate    | date                   |           |          |         | plain    |             |              | 
 shipdate     | date                   |           |          |         | plain    |             |              | 
 shipmode     | character varying(16)  |           |          |         | extended |             |              | 
 customerid   | character(8)           |           |          |         | extended |            

In [7]:
#Get Data File
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [8]:
# Date Format Issues
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "set datestyle to PostgreSQL,European; show datestyle"

   DateStyle   
---------------
 Postgres, DMY
(1 row)



In [15]:
# Load Data
# Need to use U = postgres and provide password = 'pass'
!psql -h localhost -p 5432 -U postgres -d dbpraxis -c \
"set datestyle to PostgreSQL,European;copy ss_order FROM '/content/SS_Orders.csv' DELIMITER ',' CSV HEADER;"

Password for user postgres: 
COPY 9994


In [16]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select * from ss_order limit 10'
#the date data should have loaded correctly

 rowid |    orderid     | orderdate  |  shipdate  |    shipmode    | customerid |  customername   |  segment  |    country    |      city       |   state    | postalcode | region |    productid    |    category     | subcategory |                           productname                            | sales  | quantity | discount | profit  
-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------
     1 | CA-2016-152156 | 2016-11-08 | 2016-11-11 | Second Class   | CG-12520   | Claire Gute     | Consumer  | United States | Henderson       | Kentucky   | 42420      | South  | FUR-BO-10001798 | Furniture       | Bookcases   | Bush Somerset Collection Bookcase                                | 261.96 |        2 |     0.

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [17]:
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return


In [18]:
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [20]:
runSelect('select * from ss_order limit 3')

rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,state,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.00,219.58
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.00,6.87


#Group By - Basic

In [21]:
sql = "\
SELECT segment, region, category, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region, category; \
"
runSelect(sql)

segment,region,category,sales
Home Office,South,Furniture,16853.45
Corporate,South,Technology,46310.75
Corporate,West,Furniture,83080.14
Home Office,East,Furniture,29870.32
Consumer,West,Office Supplies,110080.90
Consumer,Central,Technology,72690.75
Home Office,Central,Furniture,25482.38
Corporate,South,Office Supplies,45930.19
Consumer,Central,Office Supplies,93111.40
Home Office,Central,Office Supplies,32777.22


In [22]:
sql = "\
SELECT segment, region, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region; \
"
runSelect(sql)

segment,region,sales
Corporate,Central,157995.79
Consumer,East,350908.17
Corporate,South,121885.95
Corporate,West,225855.25
Home Office,East,127463.74
Corporate,East,200409.45
Home Office,South,74255.03
Consumer,Central,252031.43
Home Office,West,136721.86
Consumer,South,195580.92


In [23]:
sql = "\
SELECT segment, SUM (sales) sales \
FROM ss_order \
GROUP BY segment; \
"
runSelect(sql)

segment,sales
Consumer,1161401.34
Corporate,706146.44
Home Office,429653.29


In [24]:
sql = "\
SELECT  SUM (sales) sales \
FROM ss_order  \
"
runSelect(sql)

sales
2297201.07


# Grouping Sets

In [25]:
#Rather confusing
sql = "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment), \
        (category,region), \
        (category, segment), \
        (segment, region), \
        (category), \
        (region), \
        (segment), \
        () \
    ); \
"
runSelect(sql)

category,region,segment,qty
None,None,None,37873
Furniture,East,Corporate,723
Office Supplies,East,Home Office,1182
Office Supplies,East,Consumer,3245
Technology,Central,Consumer,790
Technology,East,Corporate,600
Furniture,West,Home Office,476
Furniture,Central,Home Office,333
Office Supplies,South,Corporate,1281
Technology,West,Consumer,1235


In [26]:
sql = "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region), \
        (category), \
        (region), \
        () \
    ); \
"
runSelect(sql)

category,region,qty
None,None,37873
Technology,West,2335
Technology,South,1118
Office Supplies,South,3800
Office Supplies,West,7235
Furniture,Central,1827
Office Supplies,East,6462
Furniture,East,2214
Office Supplies,Central,5409
Furniture,South,1291


#Roll Up

## Basic Roll up

In [27]:
sql = "\
SELECT category, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
runSelect(sql)

category,sum
None,37873
Furniture,8028
Office Supplies,22906
Technology,6939


In [28]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
runSelect(sql)

category,sum
All category,37873
Furniture,8028
Office Supplies,22906
Technology,6939


## MultiColumn Roll Up

In [30]:
 sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (category , segment); \
"
runSelect(sql)

category,segment,quantity
None,None,37873
Furniture,Corporate,2495
Furniture,Home Office,1367
Technology,Consumer,3597
Office Supplies,Corporate,7018
Office Supplies,Consumer,11758
Technology,Corporate,2095
Technology,Home Office,1247
Office Supplies,Home Office,4130
Furniture,Consumer,4166


In [33]:
# note change in order of ROLLUP
sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
runSelect(sql)

category,segment,quantity
None,None,37873
Office Supplies,Home Office,4130
Furniture,Corporate,2495
Office Supplies,Consumer,11758
Furniture,Home Office,1367
Technology,Consumer,3597
Office Supplies,Corporate,7018
Technology,Corporate,2095
Technology,Home Office,1247
Furniture,Consumer,4166


In [34]:
sql ="\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
runSelect(sql)

category,segment,quantity
All category,All segment,37873
Office Supplies,Home Office,4130
Furniture,Corporate,2495
Office Supplies,Consumer,11758
Furniture,Home Office,1367
Technology,Consumer,3597
Office Supplies,Corporate,7018
Technology,Corporate,2095
Technology,Home Office,1247
Furniture,Consumer,4166


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)